In [6]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Define the global physical parameters for the variable mass system
rho_ini = 1020
rho_exhaust = 0.002 * rho_ini
L = 1  # m
h = L / 2  # m
R = L  # m
u = 5  # m/s
m_dot = -rho_exhaust * u * np.pi * R**2
w0 = 0.2  # rad/s

# Initial angular rates
w10 = 0  # rad/s
w20 = w0  # rad/s
w30 = 0.3  # rad/s

# Initial mass properties
mf0 = np.pi * R**2 * L * rho_ini  # kg
I10 = mf0 * (R**2 / 4 + h**2 / 3)  # kg*m^2
I30 = mf0 * R**2 / 2  # kg*m^2
chi0 = 0  # Initial precession angle
Fd0 = 0  # Initial F value
tb = -mf0 / m_dot - 1  # Burn time

# Quaternion initial conditions (representing no initial rotation)
q0 = [1, 0, 0, 0]  # (w, x, y, z)

# Combine initial conditions
Y0 = [mf0, I10, I30, w10, w20, w30, chi0, Fd0] + q0

# Define the differential equations
def uniburn(t, w):
    m, I1, I3, w1, w2, w3, chi, F, qw, qx, qy, qz = w
    # Mass varying terms
    md = -rho_exhaust * np.pi * R**2 * u  # mass
    I1d = md * (R**2 / 4 + h**2 / 3)  # transverse moment of inertia
    I3d = md * R**2 / 2  # spin moment of inertia
    # Equations of motion for the axisymmetric cylinder undergoing uniform burn
    w1d = (I1 - I3) * w2 * w3 / I1 - (I1d - md * (h**2 + R**2 / 4)) * w1 / I1
    w2d = -(I1 - I3) * w1 * w3 / I1 - (I1d - md * (h**2 + R**2 / 4)) * w2 / I1
    w3d = -(I3d - md * R**2 / 2) * w3
    chid = (1 - I3 / I1) * w3
    Fd = -(I1d - md * (h**2 + R**2 / 4)) / I1
    # Quaternion derivative
    omega_quat = np.array([0, w1, w2, w3])
    quat = np.array([qw, qx, qy, qz])
    quat_dot = 0.5 * np.array(quat_mult(quat, omega_quat))
    wd = [md, I1d, I3d, w1d, w2d, w3d, chid, Fd, quat_dot[0], quat_dot[1], quat_dot[2], quat_dot[3]]
    return wd

# Function to multiply two quaternions
def quat_mult(q, r):
    w1, x1, y1, z1 = q
    w2, x2, y2, z2 = r
    return [
        w1*w2 - x1*x2 - y1*y2 - z1*z2,
        w1*x2 + x1*w2 + y1*z2 - z1*y2,
        w1*y2 - x1*z2 + y1*w2 + z1*x2,
        w1*z2 + x1*y2 - y1*x2 + z1*w2
    ]

# Time span for the simulation
dt = 0.01
t_eval = np.arange(0, tb, dt)

# Solve the ODEs
sol = solve_ivp(uniburn, [0, tb], Y0, t_eval=t_eval, atol=1e-9, rtol=1e-8)

# Check if the integration was successful
if sol.status == 0:
    print("Integration successful.")
else:
    print(f"Integration failed with status {sol.status}: {sol.message}")

# Print the final time to check if it reached the end of the burn time
print(f"Final time: {sol.t[-1]}")

# Extract results
t = sol.t
m = sol.y[0]
I1 = sol.y[1]
I3 = sol.y[2]
omega1 = sol.y[3]
omega2 = sol.y[4]
omega3 = sol.y[5]
chi = sol.y[6]
F = sol.y[7]
qw, qx, qy, qz = sol.y[8], sol.y[9], sol.y[10], sol.y[11]


# Function to convert quaternion to Euler angles (Z-X-Z sequence)
def quat_to_euler_zxz(q):
    """
    Convert a quaternion to Euler angles (Z-X-Z sequence).
    """
    w, x, y, z = q
    # Compute the Euler angles
    psi = np.arctan2(2*(w*z + x*y), 1 - 2*(y**2 + z**2))
    theta = np.arccos(2*(w*y - z*x))
    phi = np.arctan2(2*(w*z + y*x), 1 - 2*(x**2 + y**2))
    return psi, theta, phi

# Extract Euler angles from quaternions
psi, theta, phi = np.zeros(len(qw)), np.zeros(len(qw)), np.zeros(len(qw))
for i in range(len(qw)):
    psi[i], theta[i], phi[i] = quat_to_euler_zxz([qw[i], qx[i], qy[i], qz[i]])





Integration successful.
Final time: 99.0
[[ 0.00000000e+00 -2.99984887e-04 -5.99939097e-04 ... -1.53203565e-02
  -1.52254058e-02 -1.51302427e-02]
 [ 2.00000000e-01  1.99989775e-01  1.99979099e-01 ... -1.31638393e-02
  -1.31220046e-02 -1.30795926e-02]
 [ 3.00000000e-01  3.00000000e-01  3.00000000e-01 ...  3.00000000e-01
   3.00000000e-01  3.00000000e-01]]


In [8]:
import csv

# Define the output file name
output_file = 'results.csv'

# Open the file in write mode
with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write the header
    writer.writerow(['t', 'omega1', 'omega2', 'omega3'])
    
    # Write the data
    for i in range(len(t)):
        writer.writerow([t[i], omega1[i], omega2[i], omega3[i]])

print(f"Data successfully written to {output_file}")

Data successfully written to results.csv


In [24]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from matplotlib.colors import to_hex

# Load data from CSV file
data = pd.read_csv('results.csv')
omega1 = data['omega1'].values  
omega2 = data['omega2'].values
omega3 = data['omega3'].values
t = data['t'].values

def interpolator(t_val):
    omega1_interp = np.interp(t_val, t, omega1)
    omega2_interp = np.interp(t_val, t, omega2)
    omega3_interp = np.interp(t_val, t, omega3)
    return omega1_interp, omega2_interp, omega3_interp

# Create a Plotly figure
fig = go.Figure()

# Generate a gradient of colors from dark blue to light blue
colors = [to_hex(plt.cm.Blues(i / 100)) for i in range(101)]

# Loop through t_val from 0 to 100 and plot every vector
for t_val in range(101):
    omega1_interp, omega2_interp, omega3_interp = interpolator(t_val)

    # Create a 3D vector from the interpolated values
    vector = np.array([omega1_interp, omega2_interp, omega3_interp])

    # Add a 3D scatter plot for the vector
    fig.add_trace(go.Scatter3d(
        x=[0, vector[0]], 
        y=[0, vector[1]], 
        z=[0, vector[2]], 
        mode='lines+markers',
        line=dict(color=colors[t_val], width=5),
        marker=dict(size=5, color=colors[t_val])
    ))

# Add the origin point in red
fig.add_trace(go.Scatter3d(
    x=[0], 
    y=[0], 
    z=[0], 
    mode='markers',
    marker=dict(size=8, color='red'),
    name='Origin'
))

# Set the layout of the plot with a beige background for the 3D scene
fig.update_layout(
    scene=dict(
        xaxis=dict(
            title='Omega 1 (X)',
            backgroundcolor='black',
            gridcolor='white',
            showbackground=True
        ),
        yaxis=dict(
            title='Omega 2 (Y)',
            backgroundcolor='black',
            gridcolor='white',
            showbackground=True
        ),
        zaxis=dict(
            title='Omega 3 (Z)',
            backgroundcolor='black',
            gridcolor='white',
            showbackground=True
        )
    ),
    title='3D Vectors from t=0 to t=100',
    paper_bgcolor='white',  # Set the outer background color to white
    plot_bgcolor='white'    # Set the inner background color to white
)

# Show the plot
fig.show()
